In [1]:
import numpy as np
import tensorflow as tf

# Stop words
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

# Build dictionary and sentences from corpus
def build_dataset(file_name):

    # Read corpus
    with open(file_name) as f:
        corpus_text = f.read()

    # convert to lower case
    corpus_text = corpus_text.lower()

    # raw sentences is a list of sentences.
    raw_sentences = corpus_text.split('.')
    sentences = []
    for sentence in raw_sentences:
        tokens = sentence.split()
        filtered_tokens = [token for token in tokens if not token in stop_words]
        sentences.append(filtered_tokens)

    # Build Vocab
    words = []
    for sentence in sentences: 
        for word in sentence:
            words.append(word)

    # Build dictionary
    words = set(words)
    word2int = {}
    int2word = {}
    vocab_size = len(words)
    for i,word in enumerate(words):
        word2int[word] = i
        int2word[i] = word

    return sentences, word2int, int2word

# Build examples using context window
def build_examples(sentences, word2int, window_size=2):
    
    # training examples
    examples = []
    for sentence in sentences:
        for word_index, word in enumerate(sentence):
            left_index = max(word_index - window_size, 0)
            right_index = min(word_index + window_size, len(sentence)) + 1
            for nb_word in sentence[left_index : right_index]:
                if nb_word != word:
                    examples.append([word, nb_word])

    return examples

# Convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    one_hot = np.zeros(vocab_size)
    one_hot[data_point_index] = 1
    return one_hot 

# Build training data set in one-hot encoding
def build_training_data(examples, word2int, vocab_size):
    x_train = [] # input word
    y_train = [] # output word
    for example in examples:
        x_train.append(to_one_hot(word2int[ example[0] ], vocab_size))
        y_train.append(to_one_hot(word2int[ example[1] ], vocab_size))

    x_train = np.asarray(x_train)
    y_train = np.asarray(y_train)
    return x_train, y_train

# Euclidean ditance
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

# Find closet vector
def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index

# Read corpus and build training examples
context_window = 2
sentences, word2int, int2word = build_dataset("corpus.txt")
examples = build_examples(sentences, word2int, context_window)
vocab_size = len(word2int)
x_train, y_train = build_training_data(examples, word2int, vocab_size)

# Build Tensorflow graph
embedding_dim = 5
X = tf.placeholder(tf.float32, shape=(None, vocab_size))
Y = tf.placeholder(tf.float32, shape=(None, vocab_size))

W1 = tf.Variable(tf.random_normal([vocab_size, embedding_dim]))
b1 = tf.Variable(tf.random_normal([embedding_dim])) #bias
hidden = tf.add(tf.matmul(X, W1), b1)

W2 = tf.Variable(tf.random_normal([embedding_dim, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden, W2), b2))

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y* tf.log(prediction), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy_loss)

# Create session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# train for n_iter iterations
epochs = 10000
for epoch in range(epochs):
    sess.run(train_step, feed_dict={X: x_train, Y: y_train})
    print(epoch, ' loss is : ', sess.run(cross_entropy_loss, feed_dict={X: x_train, Y: y_train}))

# Word embeddings
vectors = sess.run(W1+b1)
print(int2word[find_closest(word2int['hurricane'], vectors)])
print(int2word[find_closest(word2int['florence'], vectors)])


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0  loss is :  9.980929
1  loss is :  9.958119
2  loss is :  9.935629
3  loss is :  9.913453
4  loss is :  9.891584
5  loss is :  9.870017
6  loss is :  9.848752
7  loss is :  9.827778
8  loss is :  9.807094
9  loss is :  9.7866955
10  loss is :  9.766574
11  loss is :  9.74673
12  loss is :  9.727157
13  loss is :  9.707849
14  loss is :  9.6888075
15  loss is :  9.670022
16  loss is :  9.651493
17  loss is :  9.633216
18  loss is :  9.615186
19  loss is :  9.5974
20  loss is :  9.579854
21  loss is :  9.562544
22  loss is :  9.545467
23  loss is :  9.528622
24  loss is :  9.512
25  loss is :  9.495603
26  loss is :  9.479425
27  loss is :  9.463466
28  loss is :  9.447718
29  loss is :  9.432179
30  loss is :  9.416851
31  loss is :  9.401726
32  loss is :  9.386802
33  loss is :  9.37208
34  loss is :  9.357551
35  loss is :  9.343214
36  loss is :  9.329072
37  loss is :  9.315113
38  loss is :  9.301342
39  loss is :  9.287754
40  loss is :  9.274345
41  loss is :  9.261114
42  los

339  loss is :  8.221001
340  loss is :  8.220302
341  loss is :  8.219608
342  loss is :  8.218915
343  loss is :  8.218225
344  loss is :  8.217538
345  loss is :  8.216853
346  loss is :  8.216171
347  loss is :  8.215492
348  loss is :  8.214815
349  loss is :  8.214138
350  loss is :  8.213467
351  loss is :  8.212797
352  loss is :  8.21213
353  loss is :  8.211464
354  loss is :  8.2108
355  loss is :  8.21014
356  loss is :  8.209479
357  loss is :  8.208822
358  loss is :  8.20817
359  loss is :  8.207517
360  loss is :  8.206865
361  loss is :  8.206218
362  loss is :  8.205571
363  loss is :  8.204927
364  loss is :  8.204286
365  loss is :  8.203646
366  loss is :  8.203006
367  loss is :  8.202371
368  loss is :  8.201737
369  loss is :  8.201103
370  loss is :  8.200474
371  loss is :  8.199844
372  loss is :  8.199217
373  loss is :  8.198591
374  loss is :  8.197968
375  loss is :  8.197347
376  loss is :  8.196724
377  loss is :  8.196106
378  loss is :  8.195489
379  

680  loss is :  8.042042
681  loss is :  8.041584
682  loss is :  8.041129
683  loss is :  8.04067
684  loss is :  8.0402155
685  loss is :  8.039759
686  loss is :  8.039303
687  loss is :  8.038848
688  loss is :  8.038392
689  loss is :  8.037937
690  loss is :  8.037481
691  loss is :  8.037026
692  loss is :  8.0365715
693  loss is :  8.036117
694  loss is :  8.035665
695  loss is :  8.03521
696  loss is :  8.034753
697  loss is :  8.034302
698  loss is :  8.033848
699  loss is :  8.033395
700  loss is :  8.032941
701  loss is :  8.032488
702  loss is :  8.032035
703  loss is :  8.031582
704  loss is :  8.031129
705  loss is :  8.030678
706  loss is :  8.030225
707  loss is :  8.029775
708  loss is :  8.029323
709  loss is :  8.028871
710  loss is :  8.028419
711  loss is :  8.027969
712  loss is :  8.027517
713  loss is :  8.027068
714  loss is :  8.026616
715  loss is :  8.026166
716  loss is :  8.025714
717  loss is :  8.025267
718  loss is :  8.024815
719  loss is :  8.024365


1024  loss is :  7.8950334
1025  loss is :  7.8946333
1026  loss is :  7.894232
1027  loss is :  7.8938327
1028  loss is :  7.8934317
1029  loss is :  7.8930316
1030  loss is :  7.8926306
1031  loss is :  7.892231
1032  loss is :  7.8918314
1033  loss is :  7.8914313
1034  loss is :  7.8910317
1035  loss is :  7.890631
1036  loss is :  7.890233
1037  loss is :  7.889832
1038  loss is :  7.8894334
1039  loss is :  7.889035
1040  loss is :  7.888637
1041  loss is :  7.8882365
1042  loss is :  7.88784
1043  loss is :  7.887441
1044  loss is :  7.887043
1045  loss is :  7.8866434
1046  loss is :  7.8862457
1047  loss is :  7.885847
1048  loss is :  7.8854504
1049  loss is :  7.885052
1050  loss is :  7.8846555
1051  loss is :  7.884258
1052  loss is :  7.883861
1053  loss is :  7.883462
1054  loss is :  7.8830667
1055  loss is :  7.8826714
1056  loss is :  7.882273
1057  loss is :  7.881877
1058  loss is :  7.88148
1059  loss is :  7.8810835
1060  loss is :  7.880688
1061  loss is :  7.880

...

8699  loss is :  6.3741455
8700  loss is :  6.374037
8701  loss is :  6.373926
8702  loss is :  6.373819
8703  loss is :  6.373707
8704  loss is :  6.3735986
8705  loss is :  6.37349
8706  loss is :  6.37338
8707  loss is :  6.373272
8708  loss is :  6.3731613
8709  loss is :  6.3730516
8710  loss is :  6.3729444
8711  loss is :  6.3728347
8712  loss is :  6.3727255
8713  loss is :  6.3726153
8714  loss is :  6.372507
8715  loss is :  6.3723974
8716  loss is :  6.3722878
8717  loss is :  6.37218
8718  loss is :  6.37207
8719  loss is :  6.371961
8720  loss is :  6.3718524
8721  loss is :  6.371743
8722  loss is :  6.3716335
8723  loss is :  6.371524
8724  loss is :  6.3714156
8725  loss is :  6.371306
8726  loss is :  6.3711987
8727  loss is :  6.371088
8728  loss is :  6.3709793
8729  loss is :  6.3708715
8730  loss is :  6.3707614
8731  loss is :  6.3706517
8732  loss is :  6.370544
8733  loss is :  6.3704343
8734  loss is :  6.370326
8735  loss is :  6.3702164
8736  loss is :  6.370

9023  loss is :  6.3393154
9024  loss is :  6.33921
9025  loss is :  6.3391056
9026  loss is :  6.3389983
9027  loss is :  6.3388915
9028  loss is :  6.338787
9029  loss is :  6.3386807
9030  loss is :  6.338577
9031  loss is :  6.3384705
9032  loss is :  6.338365
9033  loss is :  6.338259
9034  loss is :  6.3381534
9035  loss is :  6.338048
9036  loss is :  6.337942
9037  loss is :  6.3378377
9038  loss is :  6.337732
9039  loss is :  6.3376274
9040  loss is :  6.3375206
9041  loss is :  6.3374147
9042  loss is :  6.337309
9043  loss is :  6.3372045
9044  loss is :  6.3370986
9045  loss is :  6.336993
9046  loss is :  6.3368883
9047  loss is :  6.336783
9048  loss is :  6.336677
9049  loss is :  6.336571
9050  loss is :  6.336466
9051  loss is :  6.336361
9052  loss is :  6.3362546
9053  loss is :  6.3361506
9054  loss is :  6.336044
9055  loss is :  6.335939
9056  loss is :  6.3358335
9057  loss is :  6.335729
9058  loss is :  6.3356233
9059  loss is :  6.335518
9060  loss is :  6.33

9344  loss is :  6.3059435
9345  loss is :  6.3058414
9346  loss is :  6.305739
9347  loss is :  6.305637
9348  loss is :  6.3055353
9349  loss is :  6.3054338
9350  loss is :  6.3053308
9351  loss is :  6.3052287
9352  loss is :  6.305126
9353  loss is :  6.305024
9354  loss is :  6.3049216
9355  loss is :  6.3048186
9356  loss is :  6.304717
9357  loss is :  6.3046155
9358  loss is :  6.3045135
9359  loss is :  6.304411
9360  loss is :  6.3043094
9361  loss is :  6.304208
9362  loss is :  6.3041058
9363  loss is :  6.3040037
9364  loss is :  6.3039007
9365  loss is :  6.303798
9366  loss is :  6.3036957
9367  loss is :  6.3035965
9368  loss is :  6.303492
9369  loss is :  6.3033905
9370  loss is :  6.303289
9371  loss is :  6.303187
9372  loss is :  6.3030844
9373  loss is :  6.3029823
9374  loss is :  6.3028812
9375  loss is :  6.3027782
9376  loss is :  6.3026776
9377  loss is :  6.302575
9378  loss is :  6.302473
9379  loss is :  6.302371
9380  loss is :  6.30227
9381  loss is :  

9665  loss is :  6.2736444
9666  loss is :  6.2735453
9667  loss is :  6.2734466
9668  loss is :  6.273348
9669  loss is :  6.2732487
9670  loss is :  6.273149
9671  loss is :  6.27305
9672  loss is :  6.272951
9673  loss is :  6.272853
9674  loss is :  6.2727537
9675  loss is :  6.272655
9676  loss is :  6.272556
9677  loss is :  6.2724566
9678  loss is :  6.2723584
9679  loss is :  6.2722597
9680  loss is :  6.2721605
9681  loss is :  6.272061
9682  loss is :  6.2719626
9683  loss is :  6.2718635
9684  loss is :  6.2717657
9685  loss is :  6.271666
9686  loss is :  6.2715673
9687  loss is :  6.271469
9688  loss is :  6.2713704
9689  loss is :  6.27127
9690  loss is :  6.2711716
9691  loss is :  6.271074
9692  loss is :  6.2709746
9693  loss is :  6.270876
9694  loss is :  6.270777
9695  loss is :  6.270679
9696  loss is :  6.27058
9697  loss is :  6.270481
9698  loss is :  6.270383
9699  loss is :  6.270285
9700  loss is :  6.270185
9701  loss is :  6.2700872
9702  loss is :  6.26998

9996  loss is :  6.241405
9997  loss is :  6.2413116
9998  loss is :  6.241213
9999  loss is :  6.241118
banks
work
